In [1]:
import scipy.io
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
mat_data_22 = scipy.io.loadmat(r'D:\New folder\Dissertation\EXIOBASE 2022.mat')

In [3]:
Z2022 = mat_data_22['Z2022']

In [4]:
countries_df = pd.read_excel(r'D:\New folder\Dissertation\49 countries  163 industries.xlsx', sheet_name='Countries', usecols="D",skiprows=2, nrows=49)
countries = countries_df.iloc[:, 0].tolist()  # Convert to plain list
sectors_df = pd.read_excel(r'D:\New folder\Dissertation\49 countries  163 industries.xlsx',sheet_name='Industries', header=None, usecols='C',skiprows=2,nrows=163 )
sectors_df.columns = ['Sector']
#print(sectors_df)

In [5]:
Y2022 = mat_data_22['Y2022']
Y2022.shape

(7987, 49)

In [6]:
# === Load Total Output from the same .mat file ===

X2022 = mat_data_22['O2022'].flatten()  # Flatten to ensure it's a 1D NumPy array

In [7]:
# === Load Value Added from the .mat file ===
VA2022 = mat_data_22['VA2022'].flatten()  # Flatten to 1D just like O2022

va_matrix = VA2022.reshape(49, 163)

# Step 3: Sum across sectors (i.e., sum each row)
country_va_totals = va_matrix.sum(axis=1)  # Shape: (49,)

# Step 4: Create a DataFrame with country names and total VA
df_va_by_country = pd.DataFrame({
    'Country': countries,
    'Total_VA_2022': country_va_totals
})

#print(df_va_by_country)

In [8]:
# === Load Carbon Emission from the .mat file ===
C2022 = mat_data_22['C2022'].flatten()  # Flatten to 1D just like O2022


In [9]:
#x_diag_22 = np.diag(X2022)
#x_diag_safe_22 = np.where(x_diag_22 == 0, 1e-10, x_diag_22)

# Step 3: Compute the inverse of the diagonal matrix element-wise (reciprocal of each diagonal element)
#x_diag_inv_22 = np.diag(1 / x_diag_safe_22.diagonal())
# Step 2: Multiply Z by the inverse of the diagonal matrix
#A22 = Z2022 @ x_diag_inv_22

#print(A22)

In [22]:
#identity_matrix = np.eye(A22.shape[0])

In [24]:
#L22=identity_matrix-A22
#L22_inv = np.linalg.inv(L22)

In [26]:
# Define the new sector names and index ranges (0-based indexing)
new_sectors = [
    ("Agriculture, Forestry and Fishing", range(0, 19)),
    ("Mining and Quarrying", list(range(19, 34)) + list(range(70, 85))),  # 0-based
    ("Manufacturing", [i for i in range(34, 95) if i not in range(70, 85)]),
    ("Electricity and Gas", range(95, 110)),
    ("Water supply, Sewerage, Waste Management and Remediation Activities", range(110, 112)),
    ("Construction", range(112, 114)),
    ("Wholesale and Retail Trade incl. Vehicle Repair", range(114, 119)),
    ("Transportation and Storage", range(119, 125)),
    ("Accommodation and Food Service Activities", [125]),
    ("Information and Communication", [126]),
    ("Financial and Insurance Activities", range(127, 130)),
    ("Real Estate Activities", [130]),
    ("Business Activities", range(131, 135)),
    ("Public Administration and Defence; Compulsory Social Security", [135]),
    ("Education", [136]),
    ("Human Health and Social Work Activities", [137]),
    ("Other Services Activities", range(138, 163)),
]

# Aggregate into new DataFrame
aggregated_va = []

# Step 2: Loop over each new sector definition and sum corresponding columns
for sector_name, col_indices in new_sectors:
    # Ensure indices are a list (in case it's a range)
    indices = list(col_indices)
    
    # Sum across the selected columns for all countries
    sector_total = va_matrix[:, indices].sum(axis=1)  # Shape: (49,)
    
    # Add to results
    aggregated_va.append(sector_total)

# Step 3: Stack all sector totals column-wise (transpose to get 49 rows)
aggregated_va_matrix = np.vstack(aggregated_va).T  # Shape: (49, len(new_sectors))

# Step 4: Create DataFrame with country names and new sector totals
sector_names = [s[0] for s in new_sectors]

df_aggregated_va = pd.DataFrame(aggregated_va_matrix, columns=sector_names)
df_aggregated_va.insert(0, 'Country', countries)  # Add country names as first column

#print(df_aggregated_va)
# Step 5: Export to Excel
#df_aggregated_va.to_excel('aggregated_va_by_sector_and_country.xlsx', index=False)

In [28]:
#top_5_gdp = df_va_by_country.sort_values(by='Total_VA_2022', ascending=False).head(5)
#print(top_5_gdp)

## Reduced Z, X, and A Matrix

In [31]:
## STUDY CASE:
country_index = countries.index("Indonesia")  # e.g. 42
sector_index = [sector[0] for sector in new_sectors].index("Mining and Quarrying")  # e.g. 16

# Calculate position in flattened vector
pos = country_index * len(new_sectors) + sector_index

# Create zero vector
deltaf_array_T = np.zeros((len(countries) * len(new_sectors), 1))

# Set only Indonesia Mining position to 5000
deltaf_array_T[pos, 0] = 5000

print(deltaf_array_T.shape)  # (833, 1)
print(f"Value at position {pos+1}: {deltaf_array_T[pos, 0]}")

(833, 1)
Value at position 716: 5000.0


In [33]:
Z_matrix = np.array(Z2022)  # shape: 7987 x 7987
M = np.zeros((163, len(new_sectors)))

for j, (_, indices) in enumerate(new_sectors):
    for i in indices:
        M[i, j] = 1

n_countries = 49
agg_sector_count = len(new_sectors)
reduced_size = n_countries * agg_sector_count

Z_reduced = np.zeros((reduced_size, reduced_size))

for i in range(n_countries):
    for j in range(n_countries):
        # Source block: country i to country j
        row_start = i * 163
        row_end = row_start + 163
        col_start = j * 163
        col_end = col_start + 163
        
        Z_block = Z_matrix[row_start:row_end, col_start:col_end]  # 163x163
        
        # Aggregate using M.T @ Z @ M
        Z_block_reduced = M.T @ Z_block @ M  # 17x17
        
        # Destination block
        r_start = i * agg_sector_count
        r_end = r_start + agg_sector_count
        c_start = j * agg_sector_count
        c_end = c_start + agg_sector_count
        
        Z_reduced[r_start:r_end, c_start:c_end] = Z_block_reduced

# Z_reduced is now 833 x 833
Z_reduced.shape
#df_Z_reduced = pd.DataFrame(Z_reduced)
#df_Z_reduced.to_excel("Z_reduced.xlsx", index=False, header=False)

(833, 833)

In [34]:
X_matrix = np.array(X2022)  # shape: 7987 x 7987
n_countries = 49
n_orig_sectors = 163
n_new_sectors = len(new_sectors)

# Reshape vector into (countries x sectors)
X_matrix_2d = X2022.reshape(n_countries, n_orig_sectors)

# Build aggregation matrix M: shape (163, 17)
M = np.zeros((n_orig_sectors, n_new_sectors))
for j, (_, indices) in enumerate(new_sectors):
    for i in indices:
        M[i, j] = 1

# Aggregate sectors: multiply original sectors by M to get reduced sectors
X_reduced_2d = X_matrix_2d @ M  # shape (49, 17)

# Flatten to vector if needed (833 x 1)
X_reduced = X_reduced_2d.flatten().reshape(-1, 1)
X_reduced.shape
#df_X_reduced = pd.DataFrame(X_reduced)
#df_X_reduced.to_excel("X_reduced.xlsx", index=False, header=False)

(833, 1)

In [35]:
# Step 3: Diagonal matrix and inverse
x_vector = X_reduced.flatten()  # shape (833,)
# Create diagonal matrix
x_diag_22_reduced = np.diag(x_vector)  # shape (833, 833)

diag_elements = np.diag(x_diag_22_reduced)

# Replace zeros with a small number to avoid division by zero
diag_elements_safe = np.where(diag_elements == 0, 1e-10, diag_elements)

# Compute the inverse diagonal elements
inv_diag_elements = 1 / diag_elements_safe

# Create the inverse diagonal matrix
x_diag_inv_22_reduced = np.diag(inv_diag_elements)

# Step 4: Compute A
A22_reduced = Z_reduced @ x_diag_inv_22_reduced
print(A22_reduced.shape)
x_diag_inv_22_reduced.shape
#df_A22_reduced = pd.DataFrame(A22_reduced)
#df_A22_reduced.to_excel("A22_reduced.xlsx", index=False, header=False)

(833, 833)


(833, 833)

In [36]:
identity_matrix_reduced = np.eye(A22_reduced.shape[0])
L22_reduced=identity_matrix_reduced-A22_reduced
L22_inv_reduced = np.linalg.inv(L22_reduced)

In [39]:
#print(df_aggregated_va)

In [41]:
va_values = df_aggregated_va.drop(columns='Country')

# Convert to numpy array (49 x 17)
va_array = va_values.values

# Flatten in row-major order (country1 all sectors, then country2 all sectors, etc.)
va_vector = va_array.flatten().reshape(-1, 1)  # shape (833, 1)

#print(va_vector)
print(va_vector.shape)  # (833, 1)

(833, 1)


In [44]:
# Value Added
int_va_22= va_vector.T @ x_diag_inv_22_reduced
#print(int_va_22.shape)
# Make sure intva is a 1D array
int_va_22 = np.asarray(int_va_22).flatten()  # shape: (n,)
# Broadcast across columns of L (multiply each column of L by corresponding intva value)
va_L =   int_va_22[np.newaxis, :]*  L22_inv_reduced  # shape: (n, n)
print(va_L.shape)
delta_va=va_L@deltaf_array_T
#print(delta_va)

(833, 833)


In [46]:
#df_int_va_22 = pd.DataFrame(int_va_22)
#df_int_va_22.to_excel("int_va_22.xlsx", index=False, header=False)
#df_va_L_22 = pd.DataFrame(va_L)
#df_va_L_22.to_excel("va_L.xlsx", index=False, header=False)

In [49]:
sectors = [s[0] for s in new_sectors]
countries_sectors = [f"{country} - {sector}" for country in countries for sector in sectors]

df_L = pd.DataFrame(L22_inv_reduced, index=countries_sectors, columns=countries_sectors)

# Show just a sample
#print(df_L)  # or use df_L.loc["Indonesia - Mining and Quarrying"]

In [54]:
delta_va_2d = delta_va.reshape(len(countries), len(new_sectors))  # (49, 17)

# Convert to DataFrame with proper labels
df_delta_va = pd.DataFrame(
    delta_va_2d,
    index=countries,
    columns=[sector[0] for sector in new_sectors]
)


df_dva_total = df_delta_va.sum(axis=1).reset_index()

# Rename columns
df_dva_total.columns = ['Country', 'Total_DVA']

#print(df_dva_total)

In [58]:
df_dva_total.to_excel("total_dva_by_country.xlsx", index=False)